## Using the provided Layers and Network
In this short tutorial we show how the proposed layers and network can be used.

### Using and Configuring Layers
Before we can use the layer, 
we need to define the types (channels and their orders) and q-space sampling schemas 
of the input and output feature maps.
For the input, these are the same as the ones of the output feature map of the previous layer.
In the first layer the q-space sampling schema is the one used in the dataset and the type is [1] (1 scalar channel)
when (raw) dMRI scans as input.

For the purpose of this example we will just hard-code these values:

In [1]:
# only 1 scalar channel
type_in = [1]  

# one scan with b=0 and the cubic sampling scheme (all 6 directions of the cube)
q_sampling_schema_in = [[0., 0., 0.], 
                        [1., 0., 0.], [-1., 0., 0.], [0., 1., 0.], 
                        [0., -1., 0.], [0., 0., 1.], [0., 0., -1.]]

# 2 scalar channels and 1 vector channel
type_out = [2, 1]

# we'll use the same sampling schema for the output, but we could instead also use a different one
q_sampling_schema_out = q_sampling_schema_in

#### pq-diff+p Layer
Let's first define a layer using the pq-diff+p kernel, 
which is based on the pq-diff and the p-space kernel.


In [2]:
from equideepdmri.layers.EquivariantPQLayer import EquivariantPQLayer

layer = EquivariantPQLayer(type_in, type_out,
                           kernel_definition="sum(pq_diff;p_space)",
                           p_kernel_size=5,
                           q_sampling_schema_in=q_sampling_schema_in,
                           q_sampling_schema_out=q_sampling_schema_out,
                           p_radial_basis_type="cosine",
                           p_radial_basis_params={"num_layers": 3, "num_units": 50})
print('Layer:', layer)
print('Input: ', layer.type_in, 'with Q:', layer.Q_in)
print('Output: ', layer.type_out, 'with Q:', layer.Q_out)
print('Kernel:', layer.kernel)

Layer: <EquivariantPQLayer (1,)->(2, 1)>
Input:  <SphericalTensorType (1,)> with Q: 7
Output:  <SphericalTensorType (2, 1)> with Q: 7
Kernel: SumKernel(
  (kernels): ModuleList(
    (0): <Kernel_PQ (φ_cos(|p|) * φ_gauss(|q_out|) * φ_gauss(|q_in|)) * Y(p-q) of type (1,) -> (2, 1) with basis size (2, 1) * 200>
    (1): <Kernel_PQ φ_cos(|p|) * Y(p) of type (1,) -> (2, 1) with basis size (2, 1) * 50>
  )
)


We defined to use the kernel size 5 in p-space.
We also defined to use the cosine radial basis function
with a 3 layer FC (and 50 units in each layer) applied to it for p-space.
The default radial basis function would be the Gaussian without a FC applied to it,
which is what is used for q-space as we did not define anything here.


#### pq-diff+q Layer
The definition of a layer using the pq-diff+q kernel is similar:

In [3]:
layer = EquivariantPQLayer(type_in, type_out,
                           kernel_definition="sum(pq_diff;q_space)",
                           p_kernel_size=5,
                           q_sampling_schema_in=q_sampling_schema_in,
                           q_sampling_schema_out=q_sampling_schema_out,
                           p_radial_basis_type="cosine",
                           p_radial_basis_params={"num_layers": 3, "num_units": 50})
print('Layer:', layer)
print('Input: ', layer.type_in, 'with Q:', layer.Q_in)
print('Output: ', layer.type_out, 'with Q:', layer.Q_out)
print('Kernel:', layer.kernel)

Layer: <EquivariantPQLayer (1,)->(2, 1)>
Input:  <SphericalTensorType (1,)> with Q: 7
Output:  <SphericalTensorType (2, 1)> with Q: 7
Kernel: SumKernel(
  (kernels): ModuleList(
    (0): <Kernel_PQ (φ_cos(|p|) * φ_gauss(|q_out|) * φ_gauss(|q_in|)) * Y(p-q) of type (1,) -> (2, 1) with basis size (2, 1) * 200>
    (1): <Kernel_PQ (φ_gauss(|q_out|) * φ_gauss(|q_in|)) * Y(q) of type (1,) -> (2, 1) with basis size (2, 1) * 4>
  )
)


#### TP-vec Layer
To define a layer using the TP-vec kernel we do the following:

In [4]:
layer = EquivariantPQLayer(type_in, type_out,
                           kernel_definition="pq_TP",
                           p_kernel_size=5,
                           q_sampling_schema_in=q_sampling_schema_in,
                           q_sampling_schema_out=q_sampling_schema_out,
                           p_radial_basis_type="cosine",
                           p_radial_basis_params={"num_layers": 3, "num_units": 50},
                           sub_kernel_selection_rule={0: [(0, 0)],
                                                      1: [(0, 1), (1, 0), (1, 1)],
                                                      2: [(2, 2)]})
print('Layer:', layer)
print('Input: ', layer.type_in, 'with Q:', layer.Q_in)
print('Output: ', layer.type_out, 'with Q:', layer.Q_out)
print('Kernel:', layer.kernel)

Layer: <EquivariantPQLayer (1,)->(2, 1)>
Input:  <SphericalTensorType (1,)> with Q: 7
Output:  <SphericalTensorType (2, 1)> with Q: 7
Kernel: <Kernel_PQ (φ_cos(|p|) * φ_gauss(|q_out|) * φ_gauss(|q_in|)) * (Y(q) x Y(p)) of type (1,) -> (2, 1) with basis size (2, 3) * 200>


where we can see that the used tuples $(l_\mathrm{filter}, l_p, l_q)$ are defined
in the `sub_kernel_selection_rule` parameter as a dict where the keys are the $l_\mathrm{filter}$
and the values are lists of pairs $(l_p, l_q)$.


#### TP$\pm$1 Layer
To define a layer using the TP$\pm$1 kernel we only adapt the `sub_kernel_selection_rule`:

In [5]:
layer = EquivariantPQLayer(type_in, type_out,
                           kernel_definition="pq_TP",
                           p_kernel_size=5,
                           q_sampling_schema_in=q_sampling_schema_in,
                           q_sampling_schema_out=q_sampling_schema_out,
                           p_radial_basis_type="cosine",
                           p_radial_basis_params={"num_layers": 3, "num_units": 50},
                           sub_kernel_selection_rule={"l_diff_to_out_max": 1})
print('Layer:', layer)
print('Input: ', layer.type_in, 'with Q:', layer.Q_in)
print('Output: ', layer.type_out, 'with Q:', layer.Q_out)
print('Kernel:', layer.kernel)

Layer: <EquivariantPQLayer (1,)->(2, 1)>
Input:  <SphericalTensorType (1,)> with Q: 7
Output:  <SphericalTensorType (2, 1)> with Q: 7
Kernel: <Kernel_PQ (φ_cos(|p|) * φ_gauss(|q_out|) * φ_gauss(|q_in|)) * (Y(q) x Y(p)) of type (1,) -> (2, 1) with basis size (4, 6) * 200>


We could also remove the `sub_kernel_selection_rule` parameter as this value is the default.


### Stacking Layers, q-Reduction, and Nonlinearities
Now let's define multiple layers, add nonlinearities, q-reduction, and then p-space only layers.
This is an architecture similar to the one used in the paper.

First start with the pq-layers. 
There is a utility function that builds an `EquivariantPQLayer` together with a nonlinearity, 
it is called `build_pq_layer`:

In [6]:
from equideepdmri.layers.layer_builders import build_pq_layer
type_in = [1]
type_out = [2, 1]

pq_layer_1 = build_pq_layer(type_in, type_out,
                            p_kernel_size=5,
                            kernel="pq_TP",
                            q_sampling_schema_in=q_sampling_schema_in,
                            q_sampling_schema_out=q_sampling_schema_out,
                            p_radial_basis_type="cosine",
                            p_radial_basis_params={"num_layers": 3, "num_units": 50},
                            sub_kernel_selection_rule={"l_diff_to_out_max": 1},
                            non_linearity_config={"tensor_non_lin":"gated", "scalar_non_lin":"swish"})
print(pq_layer_1)
print('Input: ', pq_layer_1[0].type_in, 'with Q:', pq_layer_1[0].Q_in)
print('Output before nonlinearity: ', pq_layer_1[0].type_out, 'with Q:', pq_layer_1[0].Q_out)
print('Kernel:', pq_layer_1[0].kernel)

Sequential(
  (conv): <EquivariantPQLayer (1,)->(3, 1)>
  (non_linearity): GatedBlockNonLin()
)
Input:  <SphericalTensorType (1,)> with Q: 7
Output before nonlinearity:  <SphericalTensorType (3, 1)> with Q: 7
Kernel: <Kernel_PQ (φ_cos(|p|) * φ_gauss(|q_out|) * φ_gauss(|q_in|)) * (Y(q) x Y(p)) of type (1,) -> (3, 1) with basis size (6, 6) * 200>


Note that the used `non_linearity_config` is the default so it could also be omitted.
The output before the nonlinearity has additional scalar channels (more than we defined), because these channels are needed for the gates in the non-linearity (one additional scalar channel for each non-scalar channel).

Let's define the other pq-layers:

In [7]:
type_in = type_out  # output of previous layer is input to this one
type_out = [3, 2, 1]
pq_layer_2_type_out = type_out

pq_layer_2 = build_pq_layer(type_in, type_out,
                            p_kernel_size=5,
                            kernel="pq_TP",
                            q_sampling_schema_in=q_sampling_schema_in,
                            q_sampling_schema_out=q_sampling_schema_out,
                            p_radial_basis_type="cosine",
                            p_radial_basis_params={"num_layers": 3, "num_units": 50},
                            sub_kernel_selection_rule={"l_diff_to_out_max": 1},
                            non_linearity_config={"tensor_non_lin":"gated", "scalar_non_lin":"swish"})
print(pq_layer_2)
print('Input: ', pq_layer_2[0].type_in, 'with Q:', pq_layer_2[0].Q_in)
print('Output before nonlinearity: ', pq_layer_2[0].type_out, 'with Q:', pq_layer_2[0].Q_out)
print('Kernel:', pq_layer_2[0].kernel)

Sequential(
  (conv): <EquivariantPQLayer (2, 1)->(6, 2, 1)>
  (non_linearity): GatedBlockNonLin()
)
Input:  <SphericalTensorType (2, 1)> with Q: 7
Output before nonlinearity:  <SphericalTensorType (6, 2, 1)> with Q: 7
Kernel: <Kernel_PQ (φ_cos(|p|) * φ_gauss(|q_out|) * φ_gauss(|q_in|)) * (Y(q) x Y(p)) of type (2, 1) -> (6, 2, 1) with basis size (28, 78, 45, 9) * 200>


As we now have non-scalar input and output channels, the kernel basis gets much larger and does not only have scalar and vector channels (as before) but also 45 l=2 and 9 l=3 channels (as can be seen in the basis size (29, 78, 45, 9).

Now define the q-reduction. We'll use the `QLengthWeightedAvgPool` as used in the `late` approach.
It can either be used by importing `QLengthWeightedAvgPool` from `layers.QLengthWeightedPool` or we can again use a layer builder as follows:

In [8]:
from equideepdmri.layers.layer_builders import build_q_reduction_layer

type_in = type_out
q_reduction, type_out = build_q_reduction_layer(type_in, q_sampling_schema_in, reduction='length_weighted_average')

print(q_reduction)
print(q_reduction.type_in_out)

QLengthWeightedAvgPool(
  (radial_basis): FiniteElement_RadialBasis(
    (model): FC()
  )
)
<SphericalTensorType (3, 2, 1)>


Note that besides `length_weighted_average` we could also use the unweighted `mean` or specify `conv` (as used in `gradual` q-reduction).

Now (as q-space is reduced) let's define p-space layers. Note that no kernel needs to be specified as it is always `p_space`.

In [9]:
from equideepdmri.layers.layer_builders import build_p_layer

type_out = [1, 1]
p_layer_1 = build_p_layer(type_in, type_out,
                          kernel_size=5,
                          p_radial_basis_type="cosine",
                          p_radial_basis_params={"num_layers": 3, "num_units": 50},
                          non_linearity_config={"tensor_non_lin":"gated", "scalar_non_lin":"swish"})
print(p_layer_1)
print('Input: ', p_layer_1[0].type_in, 'has Q:', p_layer_1[0].has_Q_in)
print('Output before nonlinearity: ', p_layer_1[0].type_out, 'has Q:', p_layer_1[0].has_Q_out)
print('Kernel:', p_layer_1[0].kernel, '\n')

type_in = type_out
type_out = [1]  # only 1 scalar channel as output

# don't use nonlinearity as this is the last layer
p_layer_2 = build_p_layer(type_in, type_out,
                          kernel_size=5,
                          p_radial_basis_type="cosine",
                          p_radial_basis_params={"num_layers": 3, "num_units": 50},
                          use_non_linearity=False)
print(p_layer_2) # no non-linearity => only EquivariantPLayer
print('Input: ', p_layer_2.type_in, 'has Q:', p_layer_2.has_Q_in)
print('Output before nonlinearity: ', p_layer_2.type_out, 'has Q:', p_layer_2.has_Q_out)
print('Kernel:', p_layer_2.kernel)

Sequential(
  (conv): <EquivariantPLayer (3, 2, 1)->(2, 1)>
  (non_linearity): GatedBlockNonLin()
)
Input:  <SphericalTensorType (3, 2, 1)> has Q: False
Output before nonlinearity:  <SphericalTensorType (2, 1)> has Q: False
Kernel: <Kernel_PQ φ_cos(|p|) * Y(p) of type (3, 2, 1) -> (2, 1) with basis size (8, 10, 5, 1) * 50> 

<EquivariantPLayer (1, 1)->(1,)>
Input:  <SphericalTensorType (1, 1)> has Q: False
Output before nonlinearity:  <SphericalTensorType (1,)> has Q: False
Kernel: <Kernel_PQ φ_cos(|p|) * Y(p) of type (1, 1) -> (1,) with basis size (1, 1) * 50>


### Applying the Layers
The layers can now be applied to some input feature map, where we'll use some random feature map:

In [10]:
import torch

x = torch.randn(1, 1, 7, 10, 10, 10)  # (batch_size x dim_in x Q_in x P_z x P_y x P_x)
print("Input: ", x.size()) # Channel dim: 1*1 = 1

x = pq_layer_1(x)
print("After pq-layer 1: ", x.size()) # Channel dim: 2*1 + 1*3 = 5

x = pq_layer_2(x)
print("After pq-layer 2: ", x.size()) # Channel dim: 3*1 + 2*3 + 1*5 = 14

x = q_reduction(x)
print("After q-reduction: ", x.size()) # Channel dim unchanged (14), q-dim removed

x = p_layer_1(x)
print("After p-layer 1: ", x.size()) # Channel dim: 1*1 + 1*3 = 4

x = p_layer_2(x)
print("After p-layer 2: ", x.size()) # Channel dim: 1*1 = 1

Input:  torch.Size([1, 1, 7, 10, 10, 10])
After pq-layer 1:  torch.Size([1, 5, 7, 10, 10, 10])
After pq-layer 2:  torch.Size([1, 14, 7, 10, 10, 10])
After q-reduction:  torch.Size([1, 14, 10, 10, 10])
After p-layer 1:  torch.Size([1, 4, 10, 10, 10])
After p-layer 2:  torch.Size([1, 1, 10, 10, 10])


## Using the provided Voxel-Wise Segmentation Network
As shown before, the provided equivariant layers can be stacked to build equivariant networks.
For voxel-wise prediction (e.g. voxel-wise segmentation) we included a network.
This network uses the architecture described in the paper where first pq-layers are applied, then a q-reduction, and then p-layers. This is the same structure as we defined previusly in this example with the layer builders.

In the following sections we show how the segmentation network might be used an trained.

### Preparation of the Dataset
For the purpose of this example we will use a randomly generated dataset.
This means that real learning might not be possible but it still shows how the segmentation network could be used.

In [11]:
from example.utils import RandomDMriSegmentationDataset
dataset = RandomDMriSegmentationDataset(N=10, Q=8, num_b0=2, p_size=(10, 10, 10))

Note that the `RandomDMriSegmentationDataset` contains samples with the same p-size.
This is just for simplicity of this example, in practice the `VoxelWiseSegmentationNetwork` can handle different sizes of the samples (as it is fully-convolutional). In our training we for example cropped all scans to the bounding boxes of their brain masks to save memory and speed up the training.

### Defining the Network
Now we define the network. The hyperparameters are the same as the ones used in our best model shown in the paper.

In [12]:
from equideepdmri.network.VoxelWiseSegmentationNetwork import VoxelWiseSegmentationNetwork
model = VoxelWiseSegmentationNetwork(
    q_sampling_schema_in=dataset.q_sampling_schema,
    pq_channels=[
        [7, 4]
    ],
    p_channels=[
        [20, 5],
        [10, 3],
        [5, 2],
        [1]
    ],
    pq_kernel={
        'kernel':'pq_TP',
        'p_radial_basis_type':'cosine'
    },
    p_kernel={
        'p_radial_basis_type':'cosine'
    },
    kernel_sizes=5,
    non_linearity={
        'tensor_non_lin':'gated',
        'scalar_non_lin':'swish'
    },
    q_reduction={
        'reduction':'length_weighted_average'
    }
)
print(model)

VoxelWiseSegmentationNetwork(
  (pq_layers): ModuleList(
    (0): Sequential(
      (conv): <EquivariantPQLayer (1,)->(11, 4)>
      (non_linearity): GatedBlockNonLin()
    )
  )
  (q_reduction_layer): QLengthWeightedAvgPool(
    (radial_basis): FiniteElement_RadialBasis(
      (model): FC()
    )
  )
  (p_layers): ModuleList(
    (0): Sequential(
      (conv): <EquivariantPLayer (7, 4)->(25, 5)>
      (non_linearity): GatedBlockNonLin()
    )
    (1): Sequential(
      (conv): <EquivariantPLayer (20, 5)->(13, 3)>
      (non_linearity): GatedBlockNonLin()
    )
    (2): Sequential(
      (conv): <EquivariantPLayer (10, 3)->(7, 2)>
      (non_linearity): GatedBlockNonLin()
    )
    (3): <EquivariantPLayer (5, 2)->(1,)>
  )
)


### Training the Network
Now we train the network using our random dataset. (This example may never converge as the data is random).
The following code is a simplified version of the training code we used for our paper, e.g. validation (and computing metrics), logging and saving predicted samples, saving checkpoints, and early stopping were removed for simplicity.

In [13]:
from torch import nn
from torch.utils.data.dataloader import DataLoader
from example.utils import compute_binary_label_weights

epochs = 3
dataloader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)
pos_weight = compute_binary_label_weights(dataloader)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=5.0e-03)

for epoch in range(epochs):
    for batch in iter(dataloader):
        sample_ids, x, target, brain_mask = batch['sample_id'], batch['input'], batch['target'], batch['brain_mask']

        assert brain_mask.size(0) == 1 and len(sample_ids) == 1 and target.size(0) == 1 and x.size(0) == 1, \
                        'Currently only batch-size 1 is supported'
        sample_ids = sample_ids[0]
        brain_mask = brain_mask.squeeze(0).bool()  # (Z x Y x X)
        target = target.squeeze(0)[brain_mask]  # (num_non_masked_voxels)
        # note: x is not squeezed as model expected batch dim, it is squeezed after model is applied

        optimizer.zero_grad()

        predicted_scores = model(x).squeeze(0)  # (Z x Y x X)
        predicted_scores = predicted_scores[brain_mask]  # (num_non_masked_voxels)
        loss = criterion(predicted_scores, target)
        print('Loss:', float(loss))

        loss.backward()
        optimizer.step()

Loss: 14.580911636352539
Loss: 13.333956718444824
Loss: 10.981715202331543
Loss: 9.749805450439453
Loss: 7.926243305206299
Loss: 6.447773456573486
Loss: 5.228459358215332
Loss: 4.740058422088623
Loss: 4.1125640869140625
Loss: 3.674058675765991
Loss: 3.2416303157806396
Loss: 2.5309395790100098
Loss: 2.121861219406128
Loss: 1.8830928802490234
Loss: 1.4352169036865234
Loss: 1.174275279045105
Loss: 0.8579174876213074
Loss: 0.7635501027107239
Loss: 0.64266437292099
Loss: 0.5872637033462524
Loss: 0.5563927888870239
Loss: 0.5204508900642395
Loss: 0.5092914700508118
Loss: 0.5066750645637512
Loss: 0.5145171880722046
Loss: 0.47252941131591797
Loss: 0.48094403743743896
Loss: 0.4816356301307678
Loss: 0.4658873379230499
Loss: 0.45935654640197754
